In [1]:
pip install pymc3

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# Matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
%matplotlib inline 

import seaborn as sns

# Linear Regression to verify implementation
from sklearn.linear_model import LinearRegression

# Scipy for statistics
import scipy

# PyMC3 for Bayesian Inference
import pymc3 as pm

In [3]:
#train set
df = pd.read_csv('/Users/healthcarelab/Downloads/csv_file/x_train.csv') #데이터 불러오고

In [4]:
df = df.drop(df.columns[0], axis=1)
df.head(10)

,1,2,3
0,-64.938583,202.291412,-14.451048
1,-69.058823,197.591904,-12.799682
2,-72.430664,194.533188,-11.101911
3,-75.883049,191.134109,-9.336396
4,-79.375542,187.278091,-7.509362
5,-1.704678,104.123497,-31.008192
6,2.986092,104.576439,-33.099922
7,7.369678,105.354294,-35.226910
8,11.221000,107.158379,-37.117558
9,15.004879,110.447258,-38.664536


In [5]:
# x, y 나누기
train_x = df[['1']]
train_y = df.drop('1', axis=1)
print(train_x.shape)
print(train_y.shape)

(6000, 1)
(6000, 2)


In [6]:
# y = Ax +b
from sklearn.linear_model import LinearRegression

model = LinearRegression()
train_model = model.fit(train_x, train_y)

print('Intercept:', train_model.intercept_)
print('coefficients: ', train_model.coef_)

Intercept: [258.37423027 -42.92764249]
coefficients:  [[-0.05178006]
 [-0.23682096]]


In [7]:
train_model_intercept = np.array([[258.37423027],[-42.92764249]])

print('Intercept:', train_model_intercept)
print('coefficients: ', train_model.coef_)

Intercept: [[258.37423027]
 [-42.92764249]]
coefficients:  [[-0.05178006]
 [-0.23682096]]


In [9]:
A = train_model.coef_
b = train_model_intercept


In [13]:
train_x.std()

1    87.104682
dtype: float64

In [16]:
#2. prior p(x) >> mean, precision.inv

import scipy as sp
import scipy.stats
mu = train_x.mean()
std = train_x.std()
rv = sp.stats.norm(mu, std)
p_x = rv.pdf(train_x)
p_x.shape # p(x)

(6000, 1)

In [20]:
print(np.mean(p_x))
print(np.std(p_x))

0.0031142759979477094
0.0012107515727412651


In [21]:
precision = 1/(np.std(p_x))
print(precision)

825.9332653484793


In [22]:
# P(y) 구하기
mu = train_y.mean()
std = train_y.std()
rv = sp.stats.norm(mu, std)
p_y = rv.pdf(train_y)
p_y.shape

(6000, 2)

In [24]:
print(np.mean(p_y))
print(np.std(p_y))

0.004548599353927789
0.0026149781977377953


In [39]:
cov_y = train_y.cov()
cov_y.shape

(2, 2)

In [40]:
#P(y|x)의 precision matrix
lamda = precision
_inverse = np.linalg.inv(cov_y) #precision matrix L = cov_matrix의 inverse
L = _inverse


In [43]:
print(lamda+A.T*L*A)
print(-A.T * L)
print(-L*A)
print(L)

[[825.93326564 825.9332662 ]
 [825.9332662  825.93329612]]
[[5.66552512e-06 1.64441624e-05]
 [3.59545737e-06 1.29947768e-04]]
[[5.66552512e-06 3.59545737e-06]
 [1.64441624e-05 1.29947768e-04]]
[[1.09415197e-04 6.94371077e-05]
 [6.94371077e-05 5.48717349e-04]]


In [44]:
aa = lamda+A.T*L*A
ab = -A.T*L
ba = -L*A
bb = L

In [49]:
R = np.array([[aa, ab], [ba, bb]]) # precision matrix of P(y|x)
print(R)

[[[[8.25933266e+02 8.25933266e+02]
   [8.25933266e+02 8.25933296e+02]]

  [[5.66552512e-06 1.64441624e-05]
   [3.59545737e-06 1.29947768e-04]]]


 [[[5.66552512e-06 3.59545737e-06]
   [1.64441624e-05 1.29947768e-04]]

  [[1.09415197e-04 6.94371077e-05]
   [6.94371077e-05 5.48717349e-04]]]]


In [55]:
# covariance matrix of posterior P(x|y)
temp = lamda + A.T*L*A
_inverse = np.linalg.inv(temp)
cov_matrix_xy = _inverse
cov_matrix_xy

array([[ 34054.42904678, -34054.42781302],
       [-34054.42781302,  34054.42779   ]])

In [63]:
# test data
data = pd.read_csv('/Users/healthcarelab/Downloads/csv_file/x_test.csv') #데이터 불러오고
data = data.drop(data.columns[0], axis=1)
test_x = data[['1']] #Z
test_y = data.drop('1', axis=1)

# y = Ax +b
from sklearn.linear_model import LinearRegression

model = LinearRegression()
test_model = model.fit(test_x, test_y)

print('test_coefficients: ', test_model.coef_)

test_model_intercept = np.array([[258.09661399],[-44.43320618]])

print('test_Intercept:', test_model_intercept)

test_A = train_model.coef_
test_b = train_model_intercept

test_coefficients:  [[-0.33994115]
 [-0.28090344]]
test_Intercept: [[258.09661399]
 [-44.43320618]]


In [64]:
import scipy as sp
import scipy.stats
t_mu = test_x.mean()
t_std = test_x.std()
rv = sp.stats.norm(t_mu, t_std)
t_p_x = rv.pdf(test_x)
#p_x.shape # p(x)
t_precision = 1/(np.std(t_p_x))

# P(y) 구하기
d_mu = test_y.mean()
d_std = test_y.std()
rv = sp.stats.norm(d_mu, d_std)
d_p_y = rv.pdf(test_y)
#p_y.shape
d_cov_y = test_y.cov()

#P(y|x)의 precision matrix
test_lamda = t_precision
test_inverse = np.linalg.inv(d_cov_y) #precision matrix L = cov_matrix의 inverse
test_L = test_inverse

t_aa = test_lamda+test_A.T*test_L*test_A
t_ab = -test_A.T*test_L
t_ba = -test_L*test_A
t_bb = test_L

test_R = np.array([[t_aa, t_ab], [t_ba, t_bb]]) # precision matrix of P(y|x)

# covariance matrix of posterior P(x|y)
temp_t = test_lamda + test_A.T*test_L*test_A
t_inverse = np.linalg.inv(temp_t)
t_cov_matrix_xy = t_inverse
t_cov_matrix_xy

array([[ 5157.86640155, -5157.86529833],
       [-5157.86529833,  5157.86556868]])

In [60]:
## change the hidden z, observation x,y
#train set
df = pd.read_csv('/Users/healthcarelab/Downloads/csv_file/x_train.csv') #데이터 불러오고
df = df.drop(df.columns[0], axis=1)
train_x = df[['3']] #Z
train_y = df.drop('3', axis=1)#XandY

# y = Ax +b
from sklearn.linear_model import LinearRegression

model = LinearRegression()
train_model = model.fit(train_x, train_y)

print('coefficients: ', train_model.coef_)

train_model_intercept = np.array([[-75.86260817],[240.63659842]])

print('Intercept:', train_model_intercept)

A = train_model.coef_
b = train_model_intercept

coefficients:  [[-0.90676428]
 [-0.63462032]]
Intercept: [[-75.86260817]
 [240.63659842]]


In [61]:
import scipy as sp
import scipy.stats
mu = train_x.mean()
std = train_x.std()
rv = sp.stats.norm(mu, std)
p_x = rv.pdf(train_x)
#p_x.shape # p(x)
precision = 1/(np.std(p_x))

# P(y) 구하기
mu = train_y.mean()
std = train_y.std()
rv = sp.stats.norm(mu, std)
p_y = rv.pdf(train_y)
#p_y.shape
cov_y = train_y.cov()

#P(y|x)의 precision matrix
lamda = precision
_inverse = np.linalg.inv(cov_y) #precision matrix L = cov_matrix의 inverse
L = _inverse

aa = lamda+A.T*L*A
ab = -A.T*L
ba = -L*A
bb = L

R = np.array([[aa, ab], [ba, bb]]) # precision matrix of P(y|x)

# covariance matrix of posterior P(x|y)
temp = lamda + A.T*L*A
_inverse = np.linalg.inv(temp)
cov_matrix_xy = _inverse
cov_matrix_xy

array([[ 6983.58996456, -6983.58930668],
       [-6983.58930668,  6983.59115382]])